## Importing libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html5lib
import lxml
import numpy as np

## Scraping Match summary Table 

In [6]:
url = "https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=2022%2F23;trophy=89;type=season"

In [7]:
data = requests.get(url).text

In [8]:
soup = BeautifulSoup(data, "html.parser")

In [9]:
table = soup.find('table')


In [10]:
data_table1 = pd.read_html(str(table), flavor = "bs4")
 

In [11]:
match_results = data_table1[0] # since read_html provides a list of dataframes so we have to select the required dataframe i/e [0]

In [12]:
match_results.rename({"Scorecard":"Match_id"}, axis = 1, inplace =True)

## Scraping Batting Summary Table

In [24]:
table_list = []
for x in range(35,80):
    try:
        r= requests.get(f'https://stats.espncricinfo.com/ci/engine/match/12981{x}.html')
        data = r.text
        soup = BeautifulSoup(data, 'html.parser')
        table = soup.find_all('table')
        inning = soup.find_all('span', {'class':"ds-text-title-xs ds-font-bold ds-capitalize"})
    
        
    
        data_table1 = pd.read_html(str(table[0]), flavor = "bs4")
        data_table2 = pd.read_html(str(table[2]), flavor = "bs4")
    
        #################################################################
        data_tableUpdate1 = data_table1[0]
        l1 = []
        for i in range(len(data_tableUpdate1.axes[0])):
            l1.append(inning[0].text)
        data_tableUpdate1["Innings"] = l1
        
        #################################################################
        data_tableUpdate2 = data_table2[0]
        l2 = []
        for j in range(len(data_tableUpdate2.axes[0])):
            l2.append(inning[1].text)
        data_tableUpdate2["Innings"] = l2
        
        
        ###############################################################################

        l = []
        for k in range(len(data_tableUpdate1.axes[0])):
            l.append(f'{match_results.iloc[x-35,0]} vs {match_results.iloc[x-35,1]}')
        data_tableUpdate1["MatchId"] = l
        
        l = []
        for k in range(len(data_tableUpdate2.axes[0])):
            l.append(f'{match_results.iloc[x-35,0]} vs {match_results.iloc[x-35,1]}')
        data_tableUpdate2["MatchId"] = l
        
        # storing the tables in a list 
        table_list.append(data_tableUpdate1)
        table_list.append(data_tableUpdate2)

    
    except: IndexError
        
     

### Data cleaning 

In [25]:
#Data cleaning and processing
for x in range(len(table_list)):
    try:
        table_list[x].columns = ["Batsman","Out/Not out", "Runs","Balls","M", "4s", "6s", "StrikeRate","drop1","drop2","Innings","MatchId" ]
        table_list[x].drop(["drop1","drop2"], axis = 1, inplace = True)
        table_list[x].dropna(axis = 0, inplace = True)
        table_list[x] = table_list[x].reset_index(drop = True)
        table_list[x]["BattingPos"] = table_list[x].index + 1
    except: ValueError
        
#checking the changes done
table_list[1]


,Batsman,Out/Not out,Runs,Balls,M,4s,6s,StrikeRate,Innings,MatchId,BattingPos
0,Pathum Nissanka,c Smit b Shikongo,9,10,16,1,0,90.00,Sri Lanka,Namibia vs Sri Lanka,1
1,Kusal Mendis †,c †Green b Wiese,6,6,8,0,0,100.00,Sri Lanka,Namibia vs Sri Lanka,2
2,Dhananjaya de Silva,c Shikongo b Frylinck,12,11,25,1,0,109.09,Sri Lanka,Namibia vs Sri Lanka,3
3,Danushka Gunathilaka,c †Green b Shikongo,0,1,2,0,0,0.00,Sri Lanka,Namibia vs Sri Lanka,4
4,Bhanuka Rajapaksa,c la Cock b Scholtz,20,21,39,2,0,95.23,Sri Lanka,Namibia vs Sri Lanka,5
5,Dasun Shanaka (c),c †Green b Frylinck,29,23,37,2,1,126.08,Sri Lanka,Namibia vs Sri Lanka,6
6,Wanindu Hasaranga de Silva,c Loftie-Eaton b Scholtz,4,8,9,0,0,50.00,Sri Lanka,Namibia vs Sri Lanka,7
7,Chamika Karunaratne,c Baard b Smit,5,8,15,0,0,62.50,Sri Lanka,Namibia vs Sri Lanka,8
8,Pramod Madushan,run out (van Lingen/†Green),0,0,4,0,0,-,Sri Lanka,Namibia vs Sri Lanka,9
9,Dushmantha Chameera,c Erasmus b Wiese,8,15,20,0,0,53.33,Sri Lanka,Namibia vs Sri Lanka,10


In [26]:
#merging all the tables as one table called as batting summary

data_bat_summ = pd.concat(table_list)

# changing the column values 
data_bat_summ["Out/Not out"] = np.where(data_bat_summ["Out/Not out"] == "not out","not out", "out")

# checking the changes
data_bat_summ[50:100]

    

,Batsman,Out/Not out,Runs,Balls,M,4s,6s,StrikeRate,Innings,MatchId,BattingPos
5,Michael Leask,out,4,6,8,0,0,66.66,Scotland,Scotland vs West Indies,6
6,Chris Greaves,not out,16,11,19,2,0,145.45,Scotland,Scotland vs West Indies,7
7,"Did not bat: Mark Watt, Josh Davey, Safyaan Sh...",out,"Did not bat: Mark Watt, Josh Davey, Safyaan Sh...","Did not bat: Mark Watt, Josh Davey, Safyaan Sh...","Did not bat: Mark Watt, Josh Davey, Safyaan Sh...","Did not bat: Mark Watt, Josh Davey, Safyaan Sh...","Did not bat: Mark Watt, Josh Davey, Safyaan Sh...","Did not bat: Mark Watt, Josh Davey, Safyaan Sh...",Scotland,Scotland vs West Indies,8
8,"Fall of wickets: 1-55 (Michael Jones, 6.2 ov),...",out,"Fall of wickets: 1-55 (Michael Jones, 6.2 ov),...","Fall of wickets: 1-55 (Michael Jones, 6.2 ov),...","Fall of wickets: 1-55 (Michael Jones, 6.2 ov),...","Fall of wickets: 1-55 (Michael Jones, 6.2 ov),...","Fall of wickets: 1-55 (Michael Jones, 6.2 ov),...","Fall of wickets: 1-55 (Michael Jones, 6.2 ov),...",Scotland,Scotland vs West Indies,9
0,Kyle Mayers,out,20,13,12,3,1,153.84,West Indies,Scotland vs West Indies,1
1,Evin Lewis,out,14,13,30,1,1,107.69,West Indies,Scotland vs West Indies,2
2,Brandon King,out,17,15,24,3,0,113.33,West Indies,Scotland vs West Indies,3
3,Nicholas Pooran (c)†,out,5,9,14,0,0,55.55,West Indies,Scotland vs West Indies,4
4,Shamarh Brooks,out,4,9,24,0,0,44.44,West Indies,Scotland vs West Indies,5
5,Rovman Powell,out,5,6,12,0,0,83.33,West Indies,Scotland vs West Indies,6


In [27]:
# Dropping the extra rows which are not useful
print(data_bat_summ.shape)
discard = ["Did not bat","Fall of wickets"]
data_bat_summ = data_bat_summ[~data_bat_summ.Batsman.str.contains('|'.join(discard))]
print(data_bat_summ.shape)



(846, 11)
(699, 11)


In [28]:
data_bat_summ.reset_index(drop = True, inplace = True)


In [29]:
data_bat_summ.dtypes


Batsman        object
Out/Not out    object
Runs           object
Balls          object
M              object
4s             object
6s             object
StrikeRate     object
Innings        object
MatchId        object
BattingPos      int64
dtype: object

In [30]:
#changing the type of column
data_bat_summ = data_bat_summ.astype(str)

### connecting the Match results dataframe and Batting summary dataframe by match id column

In [31]:
match_results.head()

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Match_id
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [32]:
match_results= match_results.astype(str)

In [13]:
#Connecting the dataframes match summary and batting summary using dictionary
match_id_dict = {}
for index, row in match_results.iterrows():
    key1 = row['Team 1'] + ' vs ' + row['Team 2']
    key2 = row['Team 2'] + ' vs ' + row['Team 1']
    
    match_id_dict[key1]= row['Match_id']
    match_id_dict[key2]= row['Match_id']



In [34]:
data_bat_summ["Match_id"] = data_bat_summ["MatchId"].map(match_id_dict)
data_bat_summ.head()

,Batsman,Out/Not out,Runs,Balls,M,4s,6s,StrikeRate,Innings,MatchId,BattingPos,Match_id
0,Michael van Lingen,out,3,6,7,0,0,50.00,Namibia,Namibia vs Sri Lanka,1,T20I # 1823
1,Divan la Cock,out,9,9,15,1,0,100.00,Namibia,Namibia vs Sri Lanka,2,T20I # 1823
2,Jan Nicol Loftie-Eaton,out,20,12,18,1,2,166.66,Namibia,Namibia vs Sri Lanka,3,T20I # 1823
3,Stephan Baard,out,26,24,49,2,0,108.33,Namibia,Namibia vs Sri Lanka,4,T20I # 1823
4,Gerhard Erasmus (c),out,20,24,30,0,0,83.33,Namibia,Namibia vs Sri Lanka,5,T20I # 1823


In [35]:
# Saving the dataframe to the desired location
data_bat_summ.to_csv(r'D:\CAT 2020\python\python Git projects\T20 WC analysis project\Batting_Summary.csv')
match_results.to_csv(r'D:\CAT 2020\python\python Git projects\T20 WC analysis project\Match_summary.csv')

## Scraping Bowling Table data

In [15]:
table_list = []
# scraping data from all the pages for each match 
for x in range(35,80):
    try:
        r= requests.get(f'https://stats.espncricinfo.com/ci/engine/match/12981{x}.html')
        data = r.text
        soup = BeautifulSoup(data, 'html.parser')
        table = soup.find_all('table')
        inning = soup.find_all('span', {'class':"ds-text-title-xs ds-font-bold ds-capitalize"})
    
        
    
        data_table1 = pd.read_html(str(table[1]), flavor = "bs4")
        data_table2 = pd.read_html(str(table[3]), flavor = "bs4")
    
        # Adding new column "Innings" in the data table
        data_tableUpdate1 = data_table1[0]
        l1 = []
        for i in range(len(data_tableUpdate1.axes[0])):
            l1.append(inning[1].text)
        data_tableUpdate1["Innings"] = l1
        
        # Adding new column "Innings" in the data table
        data_tableUpdate2 = data_table2[0]
        l2 = []
        for j in range(len(data_tableUpdate2.axes[0])):
            l2.append(inning[0].text)
        data_tableUpdate2["Innings"] = l2
        

        #Adding new column "Match" in the data table
        l = []
        for k in range(len(data_tableUpdate1.axes[0])):
            l.append(f'{match_results.iloc[x-35,0]} vs {match_results.iloc[x-35,1]}')
        data_tableUpdate1["Match"] = l
        
        #Adding new column "Match" in the data table
        l = []
        for k in range(len(data_tableUpdate2.axes[0])):
            l.append(f'{match_results.iloc[x-35,0]} vs {match_results.iloc[x-35,1]}')
        data_tableUpdate2["Match"] = l
        
        #Appending the two data tables for each inning in a list
        table_list.append(data_tableUpdate1)
        table_list.append(data_tableUpdate2)

    
    except: IndexError
        
     
    


## Data cleaning

In [16]:
# changing column names for each table in the table list
for x in range(len(table_list)):
    try:
        table_list[x].columns = ["Bowler","overs", "Maiden","runs","wickets", "economy", "0s", "4s","6s","wides","noBalls","bowlingTeam","Match" ]
#         table_list[x].drop(["drop1","drop2"], axis = 1, inplace = True)
#         table_list[x].dropna(axis = 0, inplace = True)
#         table_list[x]["BattingPos"] = table_list[x].index + 1
    except: ValueError
table_list[1]


,Bowler,overs,Maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,bowlingTeam,Match
0,Gerhard Erasmus,1,0,8,0,8.00,1,1,0,0,0,Namibia,Namibia vs Sri Lanka
1,David Wiese,4,0,16,2,4.00,13,1,0,0,0,Namibia,Namibia vs Sri Lanka
2,"1.4 to BKG Mendis, Straight up and Mendis fall...","1.4 to BKG Mendis, Straight up and Mendis fall...","1.4 to BKG Mendis, Straight up and Mendis fall...","1.4 to BKG Mendis, Straight up and Mendis fall...","1.4 to BKG Mendis, Straight up and Mendis fall...","1.4 to BKG Mendis, Straight up and Mendis fall...","1.4 to BKG Mendis, Straight up and Mendis fall...","1.4 to BKG Mendis, Straight up and Mendis fall...","1.4 to BKG Mendis, Straight up and Mendis fall...","1.4 to BKG Mendis, Straight up and Mendis fall...","1.4 to BKG Mendis, Straight up and Mendis fall...",Namibia,Namibia vs Sri Lanka
3,Bernard Scholtz,4,0,18,2,4.50,10,1,0,0,0,Namibia,Namibia vs Sri Lanka
4,"10.3 to PBB Rajapaksa, And Bhanuka is holes ou...","10.3 to PBB Rajapaksa, And Bhanuka is holes ou...","10.3 to PBB Rajapaksa, And Bhanuka is holes ou...","10.3 to PBB Rajapaksa, And Bhanuka is holes ou...","10.3 to PBB Rajapaksa, And Bhanuka is holes ou...","10.3 to PBB Rajapaksa, And Bhanuka is holes ou...","10.3 to PBB Rajapaksa, And Bhanuka is holes ou...","10.3 to PBB Rajapaksa, And Bhanuka is holes ou...","10.3 to PBB Rajapaksa, And Bhanuka is holes ou...","10.3 to PBB Rajapaksa, And Bhanuka is holes ou...","10.3 to PBB Rajapaksa, And Bhanuka is holes ou...",Namibia,Namibia vs Sri Lanka
5,Ben Shikongo,3,1,22,2,7.33,6,3,0,0,0,Namibia,Namibia vs Sri Lanka
6,"3.2 to P Nissanka, Got'em! Length ball on off,...","3.2 to P Nissanka, Got'em! Length ball on off,...","3.2 to P Nissanka, Got'em! Length ball on off,...","3.2 to P Nissanka, Got'em! Length ball on off,...","3.2 to P Nissanka, Got'em! Length ball on off,...","3.2 to P Nissanka, Got'em! Length ball on off,...","3.2 to P Nissanka, Got'em! Length ball on off,...","3.2 to P Nissanka, Got'em! Length ball on off,...","3.2 to P Nissanka, Got'em! Length ball on off,...","3.2 to P Nissanka, Got'em! Length ball on off,...","3.2 to P Nissanka, Got'em! Length ball on off,...",Namibia,Namibia vs Sri Lanka
7,JJ Smit,3,0,16,1,5.33,7,0,0,1,0,Namibia,Namibia vs Sri Lanka
8,"15.2 to C Karunaratne, And now it's surely ove...","15.2 to C Karunaratne, And now it's surely ove...","15.2 to C Karunaratne, And now it's surely ove...","15.2 to C Karunaratne, And now it's surely ove...","15.2 to C Karunaratne, And now it's surely ove...","15.2 to C Karunaratne, And now it's surely ove...","15.2 to C Karunaratne, And now it's surely ove...","15.2 to C Karunaratne, And now it's surely ove...","15.2 to C Karunaratne, And now it's surely ove...","15.2 to C Karunaratne, And now it's surely ove...","15.2 to C Karunaratne, And now it's surely ove...",Namibia,Namibia vs Sri Lanka
9,Jan Frylinck,4,0,26,2,6.50,10,0,2,1,0,Namibia,Namibia vs Sri Lanka


In [17]:
#merging the table list into a dataframe
data_ball_summ = pd.concat(table_list)


In [18]:
# Removing unwanted rows and checking the change
print(data_ball_summ.shape)
discard = [" ball "," to "]
data_ball_summ = data_ball_summ[~data_ball_summ.Bowler.str.contains('|'.join(discard))]
print(data_ball_summ.shape)

data_ball_summ.head()

(813, 13)
(500, 13)


,Bowler,overs,Maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,bowlingTeam,Match
0,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0,Sri Lanka,Namibia vs Sri Lanka
2,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0,Sri Lanka,Namibia vs Sri Lanka
4,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0,Sri Lanka,Namibia vs Sri Lanka
6,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0,Sri Lanka,Namibia vs Sri Lanka
8,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0,Sri Lanka,Namibia vs Sri Lanka


In [19]:
data_ball_summ = data_ball_summ.reset_index(drop = True)
data_ball_summ.head()

,Bowler,overs,Maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,bowlingTeam,Match
0,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0,Sri Lanka,Namibia vs Sri Lanka
1,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0,Sri Lanka,Namibia vs Sri Lanka
2,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0,Sri Lanka,Namibia vs Sri Lanka
3,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0,Sri Lanka,Namibia vs Sri Lanka
4,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0,Sri Lanka,Namibia vs Sri Lanka


In [20]:
#connecting the Match summary dataframe to the balling summary dataframe
data_ball_summ["match_id"] = data_ball_summ["Match"].map(match_id_dict)
data_ball_summ.head()

,Bowler,overs,Maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,bowlingTeam,Match,match_id
0,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0,Sri Lanka,Namibia vs Sri Lanka,T20I # 1823
1,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0,Sri Lanka,Namibia vs Sri Lanka,T20I # 1823
2,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0,Sri Lanka,Namibia vs Sri Lanka,T20I # 1823
3,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0,Sri Lanka,Namibia vs Sri Lanka,T20I # 1823
4,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0,Sri Lanka,Namibia vs Sri Lanka,T20I # 1823


In [21]:
#saving the dataframe to the desired location
data_ball_summ.to_csv(r'D:\CAT 2020\python\python Git projects\T20 WC analysis project\ Bowling_summary.csv')

## Scraping Players Information for each match

In [7]:
# Scraping the information for each match of the tournament
link_list=[]
name_list =[]
for x in range(35,80):
    r= requests.get(f'https://stats.espncricinfo.com/ci/engine/match/12981{x}.html')
    soup = BeautifulSoup(r.text, 'html.parser')
    table = soup.find_all('table')
    
    for i in range(0,4):
        players = table[i].find_all('a')
        for k in range(len(players)):
            link = players[k].get('href')
            link_list.append("https://www.espncricinfo.com/"+link)

In [8]:
# From the previous list of each mactch, now we scrape information for each palyer in that match
Batting_style =[]
Bowling_style =[]
Player_role = []
Team = []
Name=[]
for link in link_list:
    try:
        player_link = link
        pl_data = requests.get(player_link).text
        soup_pl = BeautifulSoup(pl_data, 'html.parser')
        table_pl = soup_pl.find('div', {'class':'ds-p-4'})
        
        name = soup_pl.title.text
        #reomoving extra title to extract the name of the player
        names = name.replace(name[-67:],'')
        ########################################################
        
        batting_style = table_pl.find('div',{'class':'ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-4 ds-mb-8'}).find_all('div')
        bowling_style = table_pl.find('div',{'class':'ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-4 ds-mb-8'}).find_all('div')
        player_role = table_pl.find('div',{'class':'ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-4 ds-mb-8'}).find_all('div')
        
        team = table_pl.find('div', {'class': 'ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-y-4'}).find_all('a')
        team =team[0].text
        
        if batting_style[3].text[0] == 'N':
            batting_style = batting_style[4].find('h5').text
            bowling_style = bowling_style[5].find('h5').text
            player_role = player_role[6].find('h5').text
        
        else:
            batting_style = batting_style[3].find('h5').text
            bowling_style = bowling_style[4].find('h5').text
            player_role = player_role[5].find('h5').text
        
        Name.append(names)
        Batting_style.append(batting_style)
        Bowling_style.append(bowling_style)
        Player_role.append(player_role) 
        Team.append(team)
        
    except: AttributeError


In [9]:
print(len(Batting_style))


1513


In [10]:
#creating a new list to store the above lists 
lst = [Name, Batting_style,Bowling_style, Player_role,Team ]

In [11]:
#creating a dataframe from the above list
player_info_df = pd.DataFrame(lst).transpose()
player_info_df.columns = ['name', 'batting_style', 'bowling_style', 'player_role', 'team']
player_info_df.head()

,name,batting_style,bowling_style,player_role,team
0,Michael van Lingen,Left hand Bat,Left arm Medium,Bowling Allrounder,Namibia
1,Divan la Cock,Right hand Bat,Legbreak,Opening Batter,Namibia
2,Jan Nicol Loftie-Eaton,Left hand Bat,"Right arm Medium, Legbreak",Batter,Namibia
3,Stephan Baard,Right hand Bat,Right arm Medium fast,Batter,Namibia
4,Gerhard Erasmus,Right hand Bat,Right arm Offbreak,Allrounder,Namibia


In [12]:
#dropping duplicate values and checking
print(player_info_df.shape)
player_info_df.drop_duplicates(inplace = True)
print(player_info_df.shape)
player_info_df


(1513, 5)
(227, 5)


,name,batting_style,bowling_style,player_role,team
0,Michael van Lingen,Left hand Bat,Left arm Medium,Bowling Allrounder,Namibia
1,Divan la Cock,Right hand Bat,Legbreak,Opening Batter,Namibia
2,Jan Nicol Loftie-Eaton,Left hand Bat,"Right arm Medium, Legbreak",Batter,Namibia
3,Stephan Baard,Right hand Bat,Right arm Medium fast,Batter,Namibia
4,Gerhard Erasmus,Right hand Bat,Right arm Offbreak,Allrounder,Namibia
...,...,...,...,...,...
1195,Heinrich Klaasen,Right hand Bat,Right arm Offbreak,Wicketkeeper,South Africa
1352,Nasum Ahmed,Left hand Bat,Slow Left arm Orthodox,Bowler,Bangladesh
1354,Ebadot Hossain,Right hand Bat,Right arm Fast medium,Bowler,Bangladesh
1381,Rishabh Pant,Left hand Bat,Wicketkeeper,Wicketkeeper Batter,India


In [14]:
#saving the dataframe to the desired location
player_info_df.to_csv(r'D:\CAT 2020\python\python Git projects\T20 WC analysis project\ Players_Info.csv')

# End